In [5]:
import cv2
import numpy as np


def drawPolyline(img, shapes, start, end, isClosed=False):
    points = []
    for i in range(start, end + 1):
        point = [shapes.part(i).x, shapes.part(i).y]
        points.append(point)
    points = np.array(points, dtype=np.int32)
    cv2.polylines(img, [points], isClosed, (255, 80, 0),
                  thickness=1, lineType=cv2.LINE_8)

def draw(img, shapes):
    drawPolyline(img, shapes, 0, 16)
    drawPolyline(img, shapes, 17, 21)
    drawPolyline(img, shapes, 22, 26)
    drawPolyline(img, shapes, 27, 30)
    drawPolyline(img, shapes, 30, 35, True)
    drawPolyline(img, shapes, 36, 41, True)
    drawPolyline(img, shapes, 42, 47, True)
    drawPolyline(img, shapes, 48, 59, True)
    drawPolyline(img, shapes, 60, 67, True)

In [8]:
!pip install easydict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import numpy as np

def ref3DModel():
    modelPoints = [[0.0, 0.0, 0.0],
                   [0.0, -330.0, -65.0],
                   [-225.0, 170.0, -135.0],
                   [225.0, 170.0, -135.0],
                   [-150.0, -150.0, -125.0],
                   [150.0, -150.0, -125.0]]
    return np.array(modelPoints, dtype=np.float64)


def ref2dImagePoints(shape):
    imagePoints = [[shape.part(30).x, shape.part(30).y],
                   [shape.part(8).x, shape.part(8).y],
                   [shape.part(36).x, shape.part(36).y],
                   [shape.part(45).x, shape.part(45).y],
                   [shape.part(48).x, shape.part(48).y],
                   [shape.part(54).x, shape.part(54).y]]
    return np.array(imagePoints, dtype=np.float64)


def cameraMatrix(fl, center):
    mat = [[fl, 1, center[0]],
                    [0, fl, center[1]],
                    [0, 0, 1]]
    return np.array(mat, dtype=np.float)

In [7]:
import os
import cv2
import sys
import dlib
import argparse
import numpy as np
import easydict

# from drawFace import draw
# import reference_world as world

PREDICTOR_PATH = os.path.join("/content/drive/MyDrive/AI_study/models", "shape_predictor_68_face_landmarks.dat")

if not os.path.isfile(PREDICTOR_PATH):
    print("[ERROR] USE models/downloader.sh to download the predictor")
    sys.exit()

# parser = argparse.ArgumentParser()

# parser.add_argument("-i", "--image",
#                     type=str,
#                     help="image location for pose estimation")

# parser.add_argument("-f", "--focal",
#                     type=float,
#                     help="Callibrated Focal Length of the camera")

# args = parser.parse_args()


args = easydict.EasyDict({ "image": 100, "focal": 20})


def main(image):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(PREDICTOR_PATH)

    while True:
        im = cv2.imread(image)

        faces = detector(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), 0)

        face3Dmodel = ref3DModel()

        for face in faces:
            shape = predictor(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), face)

            draw(im, shape)

            refImgPts = ref2dImagePoints(shape)

            height, width, channel = im.shape
            focalLength = args.focal * width
            cameraMatrix = cameraMatrix(focalLength, (height / 2, width / 2))

            mdists = np.zeros((4, 1), dtype=np.float64)

            # calculate rotation and translation vector using solvePnP
            success, rotationVector, translationVector = cv2.solvePnP(
                face3Dmodel, refImgPts, cameraMatrix, mdists)

            noseEndPoints3D = np.array([[0, 0, 1000.0]], dtype=np.float64)
            noseEndPoint2D, jacobian = cv2.projectPoints(
                noseEndPoints3D, rotationVector, translationVector, cameraMatrix, mdists)

            # draw nose line 
            p1 = (int(refImgPts[0, 0]), int(refImgPts[0, 1]))
            p2 = (int(noseEndPoint2D[0, 0, 0]), int(noseEndPoint2D[0, 0, 1]))
            cv2.line(im, p1, p2, (110, 220, 0),
                     thickness=2, lineType=cv2.LINE_AA)

            # calculating angle
            rmat, jac = cv2.Rodrigues(rotationVector)
            angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)
            
            print('*' * 80)
            print("Angle: ", angles)
            # print(f"Qx:{Qx}\tQy:{Qy}\tQz:{Qz}\t")
            x = np.arctan2(Qx[2][1], Qx[2][2])
            y = np.arctan2(-Qy[2][0], np.sqrt((Qy[2][1] * Qy[2][1] ) + (Qy[2][2] * Qy[2][2])))
            z = np.arctan2(Qz[0][0], Qz[1][0])
            print("AxisX: ", x)
            print("AxisY: ", y)
            print("AxisZ: ", z)
            print('*' * 80)

            gaze = "Looking: "
            if angles[1] < -15:
                gaze += "Left"
            elif angles[1] > 15:
                gaze += "Right"
            else:
                gaze += "Forward"
            cv2.putText(im, gaze, (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 80), 2)
            cv2.imshow("Head Pose", im)

        key = cv2.waitKey(10) & 0xFF
        if key == 27:
            cv2.imwrite(f"joye-{gaze}.jpg", im)
            break

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main(args.image)

usage: ipykernel_launcher.py [-h] [-i IMAGE] [-f FOCAL]
ipykernel_launcher.py: error: argument -f/--focal: invalid float value: '/root/.local/share/jupyter/runtime/kernel-452d4a92-f5fc-4e05-976f-4813d0c669a7.json'


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
